In [1]:
import duckdb

In [2]:
con = duckdb.connect('../datastore.duckdb')
con.sql("INSTALL httpfs; LOAD httpfs;")
con.sql("INSTALL ducklake; LOAD ducklake;")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [3]:
minioAuth = {
    'accessKey': 'minioadmin',
    'secretKey': 'miniopassword',
    'bucket': 'marketing-and-ecommerce',
    'endpoint': 'localhost:9000'
}

# --- 1. Konfigurasi MinIO (S3 API) ---
con.sql(f"SET s3_endpoint = '{minioAuth['endpoint']}';") 
con.sql(f"SET s3_access_key_id = '{minioAuth['accessKey']}';")
con.sql(f"SET s3_secret_access_key = '{minioAuth['secretKey']}';")
con.sql("SET s3_use_ssl = false;")
con.sql("SET s3_url_style = 'path';")

In [4]:
managedStoredDataPath = f"s3://{minioAuth['bucket']}/managedStoreData/" 
lakehousePath = '../datalakehouse.db'

In [5]:
con.sql(f"ATTACH 'ducklake:{lakehousePath}' AS datalakehouse (DATA_PATH '{managedStoredDataPath}');")
con.sql("CREATE SCHEMA IF NOT EXISTS datalakehouse.raw;") 
con.sql("CREATE SCHEMA IF NOT EXISTS datalakehouse.staging;")
con.sql("CREATE SCHEMA IF NOT EXISTS datalakehouse.model;") 

In [6]:
# List semua operasi CTAS yang akan dieksekusi (diasumsikan skema lama berada di katalog yang sama)
migration_queries = [
    # RAW (datamarketingseeds) -> datalakehouse.raw
    # Perbaikan: Mengganti skema lama menjadi datamarketingseeds.*
    "CREATE OR REPLACE TABLE datalakehouse.raw.campaigns AS SELECT * FROM raw.campaigns;",
    "CREATE OR REPLACE TABLE datalakehouse.raw.customers AS SELECT * FROM raw.customers;",
    "CREATE OR REPLACE TABLE datalakehouse.raw.events AS SELECT * FROM raw.events;",
    "CREATE OR REPLACE TABLE datalakehouse.raw.products AS SELECT * FROM raw.products;",
    "CREATE OR REPLACE TABLE datalakehouse.raw.transactions AS SELECT * FROM raw.transactions;",
    
    # STAGING (datamarketingclean) -> datalakehouse.staging
    # Perbaikan: Mengganti skema lama menjadi datamarketingclean.*
    "CREATE OR REPLACE TABLE datalakehouse.staging.campaigns AS SELECT * FROM datamarketingclean.campaigns;",
    "CREATE OR REPLACE TABLE datalakehouse.staging.customers AS SELECT * FROM datamarketingclean.customers;",
    "CREATE OR REPLACE TABLE datalakehouse.staging.events AS SELECT * FROM datamarketingclean.events;",
    "CREATE OR REPLACE TABLE datalakehouse.staging.products AS SELECT * FROM datamarketingclean.products;",
    "CREATE OR REPLACE TABLE datalakehouse.staging.transactions AS SELECT * FROM datamarketingclean.transactions;",
    
    # MODEL (datamodelling) -> datalakehouse.model
    # Perbaikan: Mengganti skema lama menjadi datamodelling.*
    "CREATE OR REPLACE TABLE datalakehouse.model.campaign_performance AS SELECT * FROM datamodelling.campaign_performance;"
]

# Eksekusi kueri
print("\n🔄 Mulai memigrasikan 11 tabel menggunakan CTAS (mode tunggal DB)...")
try:
    # --- Asumsi: Inisialisasi Koneksi ---
    # Jika Anda hanya menggunakan satu file database untuk semuanya:
    # con = duckdb.connect(database="localdatabase.db") 
    # con.sql("CREATE SCHEMA IF NOT EXISTS datalakehouse.raw;")
    # ... dst
    
    for query in migration_queries:
        con.execute("BEGIN;")
        con.sql(query)
        con.sql(f"CALL main.ducklake_set_commit_message('datalakehouse', 'Vincent', 'CREATE TABLE', extra_info => 'CREATE TABLE AWAL DARI DATA SQLMESH')")
        con.execute("COMMIT;")
        # Ambil nama skema.tabel untuk logging
        target_table_full = query.split(" ")[5]
        print(f"   - Migrasi berhasil: {target_table_full}")
    
    print("\n🎉 Migrasi 11 tabel selesai!")
    
    # Verifikasi jumlah total tabel
    catalog_name = "datalakehouse" # Anggap 'datalakehouse' adalah katalog Anda
    
    total_tables = con.sql(f"""
        SELECT count(*) 
        FROM information_schema.tables 
        WHERE table_catalog = '{catalog_name}' 
        AND table_schema IN ('raw', 'staging', 'model');
    """).fetchone()[0]
    
    print(f"🔎 Total tabel yang termigrasi di Katalog **{catalog_name}**: **{total_tables}**")

except Exception as e:
    print(f"❌ Terjadi kesalahan saat migrasi: {e}")

finally:
    # con.close()
    print("Koneksi ditutup (asumsi dilakukan di luar skrip ini).")


🔄 Mulai memigrasikan 11 tabel menggunakan CTAS (mode tunggal DB)...
   - Migrasi berhasil: AS


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

   - Migrasi berhasil: AS


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

   - Migrasi berhasil: AS
   - Migrasi berhasil: AS
   - Migrasi berhasil: AS
   - Migrasi berhasil: AS
   - Migrasi berhasil: AS


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

   - Migrasi berhasil: AS
   - Migrasi berhasil: AS
   - Migrasi berhasil: AS
   - Migrasi berhasil: AS

🎉 Migrasi 11 tabel selesai!
🔎 Total tabel yang termigrasi di Katalog **datalakehouse**: **11**
Koneksi ditutup (asumsi dilakukan di luar skrip ini).


In [7]:
con.close()